# Object Detection with PyTorch and Mask R-CNN 

In this tutorial, you will finetune a pre-trained [Mask R-CNN](https://arxiv.org/abs/1703.06870) model on images from the [Penn-Fudan Database for Pedestrian Detection and Segmentation](https://www.cis.upenn.edu/~jshi/ped_html/). The dataset has 170 images with 345 instances of pedestrians.

## Prerequisities

- If you are using an Azure Machine Learning Notebook VM, your environment already meets these prerequisites. Otherwise, go through the [Configuration](https://docs.microsoft.com/azure/machine-learning/how-to-configure-environment) steps to install the Azure Machine Learning Python SDK and [create an Azure ML Workspace](https://docs.microsoft.com/azure/machine-learning/how-to-manage-workspace#create-a-workspace).


In [65]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.65


## Diagnostics

Opt-in diagnostics for better experience, quality, and security in future releases.

In [66]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


## Initialize a workspace

Initialize a [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`, using the [from_config()](https://docs.microsoft.com/python/api/azureml-core/azureml.core.workspace(class)?view=azure-ml-py#from-config-path-none--auth-none---logger-none---file-name-none-) method.

In [67]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: gopalv-ws
Azure region: westus2
Subscription id: 15ae9cb6-95c1-483d-a0e3-b1a1a3b06324
Resource group: aifxdemo


## Create or attach existing Azure ML Managed Compute

You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/concept-compute-target) for training your model. In this tutorial, we use [Azure ML managed compute](https://docs.microsoft.com/azure/machine-learning/how-to-set-up-training-targets#amlcompute) for our remote training compute resource. Specifically, the below code creates a `STANDARD_NC6` GPU cluster that autoscales from 0 to 4 nodes.

**Creation of Compute takes approximately 5 minutes.** If the Aauzre ML Compute with that name is already in your workspace, this code will skip the creation process. 

As with other Azure servies, there are limits on certain resources associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/azure/machine-learning/how-to-manage-quotas) on the default limits and how to request more quota.

> Note that the below code creates GPU compute. If you instead want to create CPU compute, provide a different VM size to the `vm_size` parameter, such as `STANDARD_D2_V2`.

In [68]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpu-cluster1"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target.
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 1, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-03-03T18:59:34.532000+00:00', 'errors': None, 'creationTime': '2020-02-14T22:38:17.474022+00:00', 'modifiedTime': '2020-02-14T22:39:03.552424+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


## Train model on the remote compute

### Create a project directory
Create a directory that will contain all the code from your local machine that you will need access to on the remote resource. This includes the training script an any additional files your training script depends on.

In [69]:
import os

project_folder = './pytorch-peds'
os.makedirs(project_folder, exist_ok=True)

Possibly helpful: [this link](https://github.com/drabastomek/GTC/blob/master/SJ_2020/workshop/1_Setup/Setup.ipynb), and this sample dockerfile from Jordan:

```
FROM mcr.microsoft.com/azureml/base-gpu:intelmpi2018.3-cuda9.0-cudnn7-ubuntu16.04

# Install Horovod, temporarily using CUDA stubsddd 
RUN ldconfig /usr/local/cuda/lib64/stubs && \     
# Install AzureML SDK     
pip install --no-cache-dir azureml-defaults && \     
# Install PyTorch     
pip install --no-cache-dir tensorflow==2.0.0b1 tensorflow-gpu==2.0.0b1 keras==2.0.8 matplotlib==3.0.3 seaborn==0.9.0 requests==2.21.0 bs4==0.0.1 imageio==2.5.0 sklearn pandas==0.24.2 numpy==1.16.2 hickle==3.4.3 && \     
# Install Horovod     
pip install --no-cache-dir horovod==0.13.5 && \     ldconfig
```

### Copy training script and dependencies into project directory

In [70]:
import shutil

shutil.copy('data.py', project_folder)
shutil.copy('model.py', project_folder)
shutil.copy('script.py', project_folder)

'./pytorch-peds/script.py'

In [71]:
!git clone https://github.com/pytorch/vision.git

!git checkout v0.3.0

%cd vision
!cp references/detection/utils.py ../
!cp references/detection/transforms.py ../
!cp references/detection/coco_eval.py ../
!cp references/detection/engine.py ../
!cp references/detection/coco_utils.py ../
%cd ..

files_to_copy = ['utils', 'transforms', 'coco_eval', 'engine', 'coco_utils']
for file in files_to_copy:
    shutil.copy('./vision/references/detection/'+ file + '.py', project_folder)


fatal: destination path 'vision' already exists and is not an empty directory.
error: pathspec 'v0.3.0' did not match any file(s) known to git.
/home/gopalv/pytorch-object/vision
/home/gopalv/pytorch-object


### Download data and upload to Azure blob storage

First we download the sample dataset, and extract the images into local storage.

In [72]:
import urllib.request

from zipfile import ZipFile

data_file = './test.zip'

urllib.request.urlretrieve('https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip', data_file)
zip = ZipFile(file=data_file)
zip.extractall()
!ls PennFudanPed/

Annotation  PNGImages  PedMasks  added-object-list.txt	readme.txt


Then, we upload the data files to the datastore associated with this workspace, so that we can access them during training.

In [73]:
# get the default datastore
ds = ws.get_default_datastore()
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

ds.upload('./PennFudanPed', target_path='data', overwrite=False)

workspaceblobstore AzureBlob gopalvws3790775563 azureml-blobstore-e47496c6-9688-4277-a05b-ceb722514b9d
Uploading an estimated of 512 files
Target already exists. Skipping upload for data/added-object-list.txt
Target already exists. Skipping upload for data/readme.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00001.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00002.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00003.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00004.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00005.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00006.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00007.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00008.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00009.txt
Target already exists. Skipping upload 

Target already exists. Skipping upload for data/Annotation/PennPed00045.txt
Target already exists. Skipping upload for data/Annotation/PennPed00046.txt
Target already exists. Skipping upload for data/Annotation/PennPed00047.txt
Target already exists. Skipping upload for data/Annotation/PennPed00048.txt
Target already exists. Skipping upload for data/Annotation/PennPed00049.txt
Target already exists. Skipping upload for data/Annotation/PennPed00050.txt
Target already exists. Skipping upload for data/Annotation/PennPed00051.txt
Target already exists. Skipping upload for data/Annotation/PennPed00052.txt
Target already exists. Skipping upload for data/Annotation/PennPed00053.txt
Target already exists. Skipping upload for data/Annotation/PennPed00054.txt
Target already exists. Skipping upload for data/Annotation/PennPed00055.txt
Target already exists. Skipping upload for data/Annotation/PennPed00056.txt
Target already exists. Skipping upload for data/Annotation/PennPed00057.txt
Target alrea

Target already exists. Skipping upload for data/PNGImages/FudanPed00062.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00063.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00064.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00065.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00066.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00067.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00068.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00069.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00070.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00071.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00072.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00073.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00074.png
Target alrea

Target already exists. Skipping upload for data/PedMasks/FudanPed00011_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00012_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00013_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00014_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00015_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00016_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00017_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00018_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00019_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00020_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00021_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00022_mask.png
Target already exists. Skipping upload f

Target already exists. Skipping upload for data/PedMasks/PennPed00041_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00042_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00043_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00044_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00045_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00046_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00047_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00048_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00049_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00050_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00051_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00052_mask.png
Target already exists. Skipping upload for data/PedM

$AZUREML_DATAREFERENCE_4b3b9306386f4d139bd6f350e3e4f96c

### Register a dataset


In [74]:
from azureml.core import Dataset

datastore_paths = [(ds, 'data')]
penn_ds = Dataset.File.from_files(path=datastore_paths)
penn_ds.register(workspace=ws,
                 name='penn_ds',
                 description='Penn Fudan pedestrian data')

{
  "source": [
    "('workspaceblobstore', 'data')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "name": "penn_ds",
    "version": 1,
    "description": "Penn Fudan pedestrian data",
    "workspace": "Workspace.create(name='gopalv-ws', subscription_id='15ae9cb6-95c1-483d-a0e3-b1a1a3b06324', resource_group='aifxdemo')"
  }
}

### Create an experiment

In [75]:
from azureml.core import Experiment

experiment_name = 'pytorch-peds'
experiment = Experiment(ws, name=experiment_name)

### Specify dependencies with a custom Dockerfile

There are a number of ways to [use environments](https://docs.microsoft.com/azure/machine-learning/how-to-use-environments) for specifying dependencies during model training. In this case, we use a custom Dockerfile.

In [84]:
from azureml.core import Environment

my_env = Environment(name='maskr-docker')
my_env.docker.enabled = True
with open("dockerfiles/Dockerfile1", "r") as f:
    dockerfile_contents_of_your_base_image=f.read()
my_env.docker.base_dockerfile=dockerfile_contents_of_your_base_image 
my_env.docker.base_image = None
my_env.docker.gpu_support = True
my_env.python.interpreter_path = '/opt/miniconda/bin/python'
my_env.python.user_managed_dependencies = True



WARNING - 'gpu_support' is no longer necessary; AzureML now automatically detects and uses nvidia docker extension when it is available. It will be removed in a future release.
WARNING - 'gpu_support' is no longer necessary; AzureML now automatically detects and uses nvidia docker extension when it is available. It will be removed in a future release.


### Create a ScriptRunConfig

Use the [ScriptRunConfig](https://docs.microsoft.com/python/api/azureml-core/azureml.core.scriptrunconfig?view=azure-ml-py) class to define your run. Specify the source driectory, compute target, and environment.

In [85]:
from azureml.train.dnn import PyTorch
from azureml.core import ScriptRunConfig

# follow pattern from here: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments#use-environments-for-training

# Add training script to run config
runconfig = ScriptRunConfig(source_directory=project_folder, script="script.py")

# Attach compute target to run config
runconfig.run_config.target = cluster_name

# Uncomment the line below if you want to try this locally first
#runconfig.run_config.target = "local"

# Attach environment to run config
runconfig.run_config.environment = my_env

# Submit run 
run = experiment.submit(runconfig)
print(run)

Run(Experiment: pytorch-peds,
Id: pytorch-peds_1583273344_396fd37b,
Type: azureml.scriptrun,
Status: Starting)


In [86]:
# to get more details of your run
print(run.get_details())

{'runId': 'pytorch-peds_1583273344_396fd37b', 'target': 'gpu-cluster1', 'status': 'Starting', 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '8fc97d08-d181-47c7-a5c5-03e4629c3414', 'azureml.git.repository_uri': 'git@github.com:gvashishtha/pytorch-object.git', 'mlflow.source.git.repoURL': 'git@github.com:gvashishtha/pytorch-object.git', 'azureml.git.branch': 'docker', 'mlflow.source.git.branch': 'docker', 'azureml.git.commit': '656e64ba234a71bb7ce564c406d0ce5155bb3df3', 'mlflow.source.git.commit': '656e64ba234a71bb7ce564c406d0ce5155bb3df3', 'azureml.git.dirty': 'False'}, 'inputDatasets': [], 'runDefinition': {'script': 'script.py', 'useAbsolutePath': False, 'arguments': [], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'gpu-cluster1', 'dataReferences': {}, 'data': {}, 'jobName': None, 'maxRunDurationSeconds': None, 'nodeCount': 1, 'environment': {'name': 'maskr-docker', 'version': 'Autosave_2020-02-29T00:36:31

In [87]:
from azureml.widgets import RunDetails

RunDetails(run).show()
run.wait_for_completion(show_output=True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

RunId: pytorch-peds_1583273344_396fd37b
Web View: https://mlworkspace.azure.ai/portal/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/aifxdemo/providers/Microsoft.MachineLearningServices/workspaces/gopalv-ws/experiments/pytorch-peds/runs/pytorch-peds_1583273344_396fd37b

Streaming azureml-logs/55_azureml-execution-tvmps_7c329628f3ed87cd506b51138be56729b6a1aa785f383b471d3fdc7d9f6fd2c8_d.txt

2020-03-03T22:10:13Z Starting output-watcher...
2020-03-03T22:10:13Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_9125fd9b495cfdec8f7bf56c6d28d91d
Digest: sha256:9d52d6d75a74e3891366cc9505dd2bd6e6e1e68766b6167933cce9bd8cbb823a
Status: Image is up to date for gopalvwsdebb3b12.azurecr.io/azureml/azureml_9125fd9b495cfdec8f7bf56c6d28d91d:latest
a468fb0b114d610d8be4447ff57a20f085b7efb165c2076bd4372d29a78c906f
2020/03/03 22:10:20 Version: 3.0.01140.0002 Branch: merge Commit: 93399ed1
2020/03/0

WARNING - Retrying (Retry(total=2, connect=2, read=3, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f14f8db85c0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',)': /history/v1.0/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/aifxdemo/providers/Microsoft.MachineLearningServices/workspaces/gopalv-ws/experiments/pytorch-peds/runs/pytorch-peds_1583273344_396fd37b/details
WARNING - Retrying (Retry(total=1, connect=1, read=3, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f14f8db8278>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',)': /history/v1.0/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/aifxdemo/providers/Microsoft.MachineLearningServices/workspaces/gopalv-ws/experiments/pytorch-peds/runs/pytor

ClientRequestError: Error occurred in request., ConnectionError: HTTPSConnectionPool(host='westus2.experiments.azureml.net', port=443): Max retries exceeded with url: /history/v1.0/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/aifxdemo/providers/Microsoft.MachineLearningServices/workspaces/gopalv-ws/experiments/pytorch-peds/runs/pytorch-peds_1583273344_396fd37b/details (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f14bcb32cf8>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))

In [ ]:
from azureml.core import Run

run = Run(run_id='pytorch-peds_1583262435_ac3ea423', experiment=experiment)
model = run.register_model(model_name='pytorch_peds', model_path='outputs/model.pt')
model


In [ ]:
import torch
path = model.download(target_dir='.', exist_ok=True)
path

# model = torch.load(path)
#torch.load(model.get_model_path(model_name='outputs/model.pt'))

In [ ]:
import torch
from azureml.core import Dataset
from data import PennFudanDataset
from script import get_transform

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model = torch.load('./model.pt', map_location=device)

penn_ds = Dataset.get_by_name(workspace=ws, name='penn_ds')
dataset_test = PennFudanDataset(penn_ds, get_transform(train=False))



# pick one image from the test set
img, _ = dataset_test[0]
# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])

In [ ]:
for run in list_runs:
    print(run.get_details())

In [ ]:
from azureml.train.estimator import Estimator

script_params = {
    '--num_epochs': 10,
    '--output_dir': './outputs'
}

estimator = Estimator(source_directory=project_folder, 
                      script_params=script_params,
                      compute_target=compute_target,
                      entry_script='script.py',
                      use_gpu=True,
                      conda_dependencies_file='conda-dependencies.yml')

In [ ]:
run = experiment.submit(estimator)
print(run.get_details())

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()
run.wait_for_completion(show_output=True)

In [ ]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image


class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transforms=None):
        self.dataset = dataset
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        with self.dataset.mount() as mount_context:
            self.imgs = list(sorted(os.listdir(os.path.join(mount_context.mount_point, "PNGImages"))))
            self.masks = list(sorted(os.listdir(os.path.join(mount_context.mount_point, "PedMasks"))))

    def __getitem__(self, idx):
        # load images ad masks
        with self.dataset.mount() as mount_context:
            img_path = os.path.join(mount_context.mount_point, "PNGImages", self.imgs[idx])
            mask_path = os.path.join(mount_context.mount_point, "PedMasks", self.masks[idx])
            img = Image.open(img_path).convert("RGB")
            # note that we haven't converted the mask to RGB,
            # because each color corresponds to a different instance
            # with 0 being background
            mask = Image.open(mask_path)

            mask = np.array(mask)
            # instances are encoded as different colors
            obj_ids = np.unique(mask)
            # first id is the background, so remove it
            obj_ids = obj_ids[1:]

            # split the color-encoded mask into a set
            # of binary masks
            masks = mask == obj_ids[:, None, None]

            # get bounding box coordinates for each mask
            num_objs = len(obj_ids)
            boxes = []
            for i in range(num_objs):
                pos = np.where(masks[i])
                xmin = np.min(pos[1])
                xmax = np.max(pos[1])
                ymin = np.min(pos[0])
                ymax = np.max(pos[0])
                boxes.append([xmin, ymin, xmax, ymax])

            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            # there is only one class
            labels = torch.ones((num_objs,), dtype=torch.int64)
            masks = torch.as_tensor(masks, dtype=torch.uint8)

            image_id = torch.tensor([idx])
            area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
            # suppose all instances are not crowd
            iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

            target = {}
            target["boxes"] = boxes
            target["labels"] = labels
            target["masks"] = masks
            target["image_id"] = image_id
            target["area"] = area
            target["iscrowd"] = iscrowd

            if self.transforms is not None:
                img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
!pip install --upgrade torchvision

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

      
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [ ]:
import sys
sys.path.insert(0,'./')

In [ ]:
from platform import python_version

print(python_version())
help("modules")

In [ ]:
# from engine import train_one_epoch, evaluate
# import utils
# import transforms as T

# def get_transform(train):
#     transforms = []
#     # converts the image, a PIL image, into a PyTorch Tensor
#     transforms.append(T.ToTensor())
#     if train:
#         # during training, randomly flip the training images
#         # and ground-truth for data augmentation
#         transforms.append(T.RandomHorizontalFlip(0.5))
#     return T.Compose(transforms)


In [ ]:
# # use our dataset and defined transformations
# dataset = PennFudanDataset(penn_ds, get_transform(train=True))
# dataset_test = PennFudanDataset(penn_ds, get_transform(train=False))

# # split the dataset in train and test set
# torch.manual_seed(1)
# indices = torch.randperm(len(dataset)).tolist()
# dataset = torch.utils.data.Subset(dataset, indices[:-50])
# dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# # define training and validation data loaders
# data_loader = torch.utils.data.DataLoader(
#     dataset, batch_size=2, shuffle=True, num_workers=4,
#     collate_fn=utils.collate_fn)

# data_loader_test = torch.utils.data.DataLoader(
#     dataset_test, batch_size=1, shuffle=False, num_workers=4,
#     collate_fn=utils.collate_fn)

In [ ]:
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# # our dataset has two classes only - background and person
# num_classes = 2

# # get the model using our helper function
# model = get_instance_segmentation_model(num_classes)
# # move model to the right device
# model.to(device)

# # construct an optimizer
# params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(params, lr=0.005,
#                             momentum=0.9, weight_decay=0.0005)

# # and a learning rate scheduler which decreases the learning rate by
# # 10x every 3 epochs
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
#                                                step_size=3,
#                                                gamma=0.1)

In [ ]:
# num_epochs = 10

# for epoch in range(num_epochs):
#     # train for one epoch, printing every 10 iterations
#     train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
#     # update the learning rate
#     lr_scheduler.step()
#     # evaluate on the test dataset
#     evaluate(model, data_loader_test, device=device)

In [ ]:
# # pick one image from the test set
# img, _ = dataset_test[0]
# # put the model in evaluation mode
# model.eval()
# with torch.no_grad():
#     prediction = model([img.to(device)])

In [ ]:
# Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())